In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [8]:

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amank\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amank\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
105,FIVE STAR FINAL was one of the best films of t...,positive
323,A hilarious and insightful perspective of the ...,positive
795,This film fails on every count. For a start it...,negative
923,SWEET SIXTEEN (1983) **/***** 86 minutes Direc...,negative
137,"This film is a tapestry, a series of portraits...",positive


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
105,five star final one best film early s starred ...,positive
323,hilarious insightful perspective dating world ...,positive
795,film fails every count start pretentious striv...,negative
923,sweet sixteen minute director jim sotos cast b...,negative
137,film tapestry series portrait rom community wo...,positive


In [10]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [11]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [12]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
105,five star final one best film early s starred ...,1
323,hilarious insightful perspective dating world ...,1
795,film fails every count start pretentious striv...,0
923,sweet sixteen minute director jim sotos cast b...,0
137,film tapestry series portrait rom community wo...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/itzayush21/IMDB_MLOPS.mlflow')
dagshub.init(repo_owner='itzayush21', repo_name='IMDB_MLOPS', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as itzayush21

Initialized MLflow to track repo "itzayush21/IMDB_MLOPS"

Repository itzayush21/IMDB_MLOPS initialized!

2025/08/17 15:45:48 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e0b114399d8444bc97c1c10eeff22979', creation_time=1755425750796, experiment_id='0', last_update_time=1755425750796, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-17 15:48:31,547 - INFO - Starting MLflow run...
2025-08-17 15:48:32,832 - INFO - Logging preprocessing parameters...
2025-08-17 15:48:34,004 - INFO - Initializing Logistic Regression model...
2025-08-17 15:48:34,006 - INFO - Fitting the model...
2025-08-17 15:48:34,078 - INFO - Model training complete.
2025-08-17 15:48:34,079 - INFO - Logging model parameters...
2025-08-17 15:48:34,466 - INFO - Making predictions...
2025-08-17 15:48:34,467 - INFO - Calculating evaluation metrics...
2025-08-17 15:48:34,492 - INFO - Logging evaluation metrics...
2025-08-17 15:48:39,280 - INFO - Saving and logging the model...
2025/08/17 15:48:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-08-17 15:48:40,562 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\amank\AppData\Local\Temp\ipykernel_19968\3610753014.py", line 48, in 

🏃 View run enchanting-mule-229 at: https://dagshub.com/itzayush21/IMDB_MLOPS.mlflow/#/experiments/0/runs/62843dc7097545ce96c469a833c1f81f
🧪 View experiment at: https://dagshub.com/itzayush21/IMDB_MLOPS.mlflow/#/experiments/0
